In [2]:
from bs4 import BeautifulSoup
import urllib
from string import replace
import numpy as np
import csv

In [3]:
def parse_table(table):
    return [
        [cell.get_text().strip() for cell in row.find_all(['th', 'td'])]
           for row in table.find_all('tr')
    ]

In [4]:
headings = []

for k in range(30):
    day = k+1
    url = 'http://cdec2.cdec-sing.cl/pls/portal/cdec.pck_programacion_pub.dsp_prog_diaria_2?\
    p_fecha='+str(day)+'-9-2016&p_estado=P'
    html_doc = urllib.urlopen(url).read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    x = parse_table(soup)  
    
    for i in [10,4,3,2,1,0]:
            x.remove(x[i])

    for i in range(6):
        last_row = len(x)-1
        x.remove(x[last_row])

    for i in range(len(x)):
        if len(x[i])==1:
            x.remove(x[i])
    
    for i in range(len(x)):
        if x[i][0] not in headings:
            headings.append(x[i][0])  

for i in range(len(headings)):
    headings[i]=headings[i].encode('utf-8')

In [6]:
duration = 30
data_array = np.zeros(shape=[24*(duration),len(headings)])

for k in range(duration):
    day = k+1
    url = 'http://cdec2.cdec-sing.cl/pls/portal/cdec.pck_programacion_pub.dsp_prog_diaria_2?\
    p_fecha='+str(day)+'-9-2016&p_estado=P'
    html_doc = urllib.urlopen(url).read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    x = parse_table(soup)  

    for i in [10,4,3,2,1,0]:
            x.remove(x[i])

    for i in range(6):
        last_row = len(x)-1
        x.remove(x[last_row])

    for i in range(len(x)):
        if len(x[i])==1:
            x.remove(x[i])

    for i in range(len(x)):
        for j in range(len(x[i])):
            if x[i][j]=='':
                x[i][j]=(str(x[i][j].replace('','0')))

    for i in range(len(x)):
        for j in range(len(x[i])):
            try:
                x[i][j]=float(str(x[i][j].replace(',','.')))
            except:
                next

    for i in range(len(x)):
        if len(x[i])==1:
            x.remove(x[i]) 
            
    for i in range(len(x)):
        x[i][0] = x[i][0].encode('utf-8')

    pointer = []
    for i in range(len(x)):
        pointer.append(headings.index(x[i][0]))
        x[i].pop(0)

    for i in range(len(x)):
        for j in range(len(x[0])-1):
            data_array[(j+(24*k)),pointer[i]]=x[i][j]

In [233]:
with open('CDEC_SING_Septiembre.csv','wb')as f:
    w = csv.writer(f,delimiter=',',lineterminator='\n')
    w.writerow(headings)
    w.writerows(data_array)